# Shor's Algorithm (Manual Implementation)

This notebook demonstrates a manual implementation of Shor's Algorithm using Qiskit without relying on the built-in `Shor` class.

In [ ]:
from qiskit import QuantumCircuit, Aer, execute
from qiskit.circuit.library import QFT
from math import gcd
from fractions import Fraction
import numpy as np


In [ ]:
def qpe_modexp(a: int, N: int, n_count: int) -> QuantumCircuit:
    qc = QuantumCircuit(n_count + 4, n_count)
    for q in range(n_count):
        qc.h(q)
    qc.x(n_count)
    for q in range(n_count):
        qc = apply_c_amod15(qc, a**(2**q) % N, q, n_count)
    qc.append(QFT(num_qubits=n_count, inverse=True, do_swaps=True), range(n_count))
    qc.measure(range(n_count), range(n_count))
    return qc


In [ ]:
def apply_c_amod15(qc, a: int, control_qubit: int, n_count: int) -> QuantumCircuit:
    x = n_count
    if a == 2:
        qc.cswap(control_qubit, x, x+1)
        qc.cswap(control_qubit, x+1, x+2)
        qc.cswap(control_qubit, x+2, x+3)
    elif a == 4:
        qc.cswap(control_qubit, x, x+2)
        qc.cswap(control_qubit, x+1, x+3)
    elif a == 7:
        qc.cswap(control_qubit, x, x+3)
        qc.cswap(control_qubit, x+1, x+2)
    elif a == 8:
        qc.cswap(control_qubit, x, x+2)
        qc.cswap(control_qubit, x+1, x+3)
        qc.cswap(control_qubit, x+2, x+3)
    elif a == 11:
        qc.cswap(control_qubit, x, x+1)
        qc.cswap(control_qubit, x+1, x+2)
        qc.cswap(control_qubit, x+2, x+3)
    elif a == 13:
        qc.cswap(control_qubit, x, x+2)
        qc.cswap(control_qubit, x+1, x+3)
        qc.cswap(control_qubit, x+2, x+3)
    return qc


In [ ]:
def get_period(phase: int, n_count: int) -> int:
    decimal = phase / (2 ** n_count)
    frac = Fraction(decimal).limit_denominator(15)
    return frac.denominator


In [ ]:
def shor_manual(N: int, a: int = 7) -> int:
    if gcd(a, N) != 1:
        return gcd(a, N)

    n_count = 8
    qc = qpe_modexp(a, N, n_count)
    backend = Aer.get_backend('qasm_simulator')
    result = execute(qc, backend, shots=1).result()
    counts = result.get_counts()
    phase_bin = max(counts, key=counts.get)
    phase_int = int(phase_bin, 2)
    r = get_period(phase_int, n_count)
    if r % 2 != 0:
        return None
    plus = pow(a, r // 2) + 1
    minus = pow(a, r // 2) - 1
    factor1 = gcd(plus, N)
    factor2 = gcd(minus, N)
    if factor1 == 1 or factor1 == N:
        return None
    return factor1


In [ ]:
N = 15
a = 7
factor = shor_manual(N, a)
print(f"Found factor of {N} using base {a}: {factor}")
